In [ ]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime

# Selenium으로 브라우저 열기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 수집할 데이터 저장할 리스트
all_recruits = []

# 현재 연도를 가져오기
current_year = datetime.now().year

# 페이지 순회
for i in range(1, 2):  # 페이지 1부터 1까지 (필요시 범위를 변경)
    url = "https://www.catch.co.kr/NCS/RecruitSearch?page={}".format(i)
    driver.get(url)

    try:
        # 특정 요소가 나타날 때까지 기다림
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'tr td p.date'))
        )
    except Exception as e:
        print(f"페이지 {i}에서 요소를 찾을 수 없습니다: {e}")
        continue

    # page_source로 페이지의 HTML 가져오기
    html = driver.page_source

    # BeautifulSoup을 사용해 HTML 파싱
    soup = BeautifulSoup(html, 'html.parser')

    # 회사명 추출
    company_names = soup.select('tr td.al1 p.name')
    # 공고문 제목 추출
    job_titles = soup.select('tr td a.link')
    # 마감일 추출
    end_dates = soup.select('tr td p.date')

    # 결과 저장 및 출력
    for company, title, date in zip(company_names, job_titles, end_dates):
        # 마감일에서 "~" 이후의 날짜만 추출
        full_date_text = date.get_text().strip()
        if "~" in full_date_text:
            end_date_str = full_date_text.split("~")[-1].strip()  # "~" 이후의 날짜만 추출
        else:
            end_date_str = full_date_text  # "~"가 없는 경우 전체 날짜 사용
        
        # 현재 연도 추가 후 날짜 형식 변환
        try:
            end_date_with_year = f"{current_year}.{end_date_str}"
            end_date = datetime.strptime(end_date_with_year, '%Y.%m.%d').strftime('%y.%m.%d')
        except ValueError:
            end_date = end_date_str  # 형식 변환 실패 시 원본 유지

        # 상세 URL 추출 (job_titles에서 href 가져오기)
        detail_url = title['href']  # title 객체에서 href 속성 가져오기
        detail_url = "https://www.catch.co.kr" + detail_url  # 전체 URL 형식으로 변환

        # 결과를 리스트에 추가
        all_recruits.append({
            "title": title.get_text().strip(),  # 공고 제목
            "company_name": company.get_text().strip(),  # 회사 이름
            "detail_url": detail_url,  # 디테일 페이지 URL
            "end_date": end_date,  # 마감일 (현재 연도 포함)
            "platform_name": "Catch"  # 플랫폼 이름
        })

# 드라이버 종료
driver.quit()

# JSON 파일로 저장
file_path = r"C:\Users\jeong\바탕 화면\catch.json"
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(all_recruits, json_file, ensure_ascii=False, indent=4)

print(f"데이터가 {file_path}에 저장되었습니다.")